***GENERATED CODE FOR telcochurnpbi PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CustomerChurn[customerID]', 'transformation_label': 'String Indexer'}], 'feature': 'CustomerChurn[customerID]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': '0021-IKXGC', 'max': '9992-UJOEL', 'missing': '0', 'distinct': '500'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CustomerChurn[customerID]'}, {'feature_label': 'CustomerChurn[customerID]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'CustomerChurn[customerID]')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CustomerChurn[gender]', 'transformation_label': 'String Indexer'}], 'feature': 'CustomerChurn[gender]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Female', 'max': 'Male', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CustomerChurn[gender]'}, {'feature_label': 'CustomerChurn[gender]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CustomerChurn[gender]')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CustomerChurn[MultipleLines]', 'transformation_label': 'String Indexer'}], 'feature': 'CustomerChurn[MultipleLines]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CustomerChurn[MultipleLin...'}, {'feature_label': 'CustomerChurn[MultipleLines]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'CustomerChurn[MultipleLines]')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CustomerChurn[InternetService]', 'transformation_label': 'String Indexer'}], 'feature': 'CustomerChurn[InternetService]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'DSL', 'max': 'Fiber optic', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CustomerChurn[InternetSer...'}, {'feature_label': 'CustomerChurn[InternetService]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'CustomerChurn[InternetService]')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CustomerChurn[OnlineSecurity]', 'transformation_label': 'String Indexer'}], 'feature': 'CustomerChurn[OnlineSecurity]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'No', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CustomerChurn[OnlineSecur...'}, {'feature_label': 'CustomerChurn[OnlineSecurity]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'CustomerChurn[OnlineSecurity]')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CustomerChurn[OnlineBackup]', 'transformation_label': 'String Indexer'}], 'feature': 'CustomerChurn[OnlineBackup]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CustomerChurn[OnlineBacku...'}, {'feature_label': 'CustomerChurn[OnlineBackup]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'CustomerChurn[OnlineBackup]')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CustomerChurn[DeviceProtection]', 'transformation_label': 'String Indexer'}], 'feature': 'CustomerChurn[DeviceProtection]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'No', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CustomerChurn[DeviceProte...'}, {'feature_label': 'CustomerChurn[DeviceProtection]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'CustomerChurn[DeviceProtection]')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CustomerChurn[TechSupport]', 'transformation_label': 'String Indexer'}], 'feature': 'CustomerChurn[TechSupport]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'No', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CustomerChurn[TechSupport...'}, {'feature_label': 'CustomerChurn[TechSupport]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'CustomerChurn[TechSupport]')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CustomerChurn[StreamingTV]', 'transformation_label': 'String Indexer'}], 'feature': 'CustomerChurn[StreamingTV]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CustomerChurn[StreamingTV...'}, {'feature_label': 'CustomerChurn[StreamingTV]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'CustomerChurn[StreamingTV]')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CustomerChurn[StreamingMovies]', 'transformation_label': 'String Indexer'}], 'feature': 'CustomerChurn[StreamingMovies]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CustomerChurn[StreamingMo...'}, {'feature_label': 'CustomerChurn[StreamingMovies]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'CustomerChurn[StreamingMovies]')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CustomerChurn[Contract]', 'transformation_label': 'String Indexer'}], 'feature': 'CustomerChurn[Contract]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Month-to-month', 'max': 'Month-to-month', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CustomerChurn[Contract]'}, {'feature_label': 'CustomerChurn[Contract]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CustomerChurn[Contract]')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CustomerChurn[PaymentMethod]', 'transformation_label': 'String Indexer'}], 'feature': 'CustomerChurn[PaymentMethod]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Bank transfer (automatic)', 'max': 'Mailed check', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CustomerChurn[PaymentMeth...'}, {'feature_label': 'CustomerChurn[PaymentMethod]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'CustomerChurn[PaymentMethod]')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run telcochurnpbiHooks.ipynb
try:
	#sourcePreExecutionHook()

	telcochurnpbihdfs = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/extra/tempFile1692360590.csv', 'filename': 'tempFile1692360590.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sourcePostExecutionHook(telcochurnpbihdfs)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run telcochurnpbiHooks.ipynb
try:
	#transformationPreExecutionHook()

	telcochurnpbiautofe = TransformationMain.run(telcochurnpbihdfs,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "CustomerChurn[customerID]", "transformation_label": "String Indexer"}], "feature": "CustomerChurn[customerID]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "0021-IKXGC", "max": "9992-UJOEL", "missing": "0", "distinct": "500"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CustomerChurn[customerID]"}, {"transformationsData": [{"feature_label": "CustomerChurn[gender]", "transformation_label": "String Indexer"}], "feature": "CustomerChurn[gender]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Female", "max": "Male", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CustomerChurn[gender]"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CustomerChurn[SeniorCitizen]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.2", "stddev": "0.4", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "CustomerChurn[SeniorCitiz..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CustomerChurn[Partner]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.22", "stddev": "0.41", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "CustomerChurn[Partner]"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CustomerChurn[Dependents]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.15", "stddev": "0.36", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "CustomerChurn[Dependents]"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CustomerChurn[tenure]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "6.53", "stddev": "12.57", "min": "1", "max": "67", "missing": "0"}, "updatedLabel": "CustomerChurn[tenure]"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CustomerChurn[PhoneService]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.81", "stddev": "0.39", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "CustomerChurn[PhoneServic..."}, {"transformationsData": [{"feature_label": "CustomerChurn[MultipleLines]", "transformation_label": "String Indexer"}], "feature": "CustomerChurn[MultipleLines]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CustomerChurn[MultipleLin..."}, {"transformationsData": [{"feature_label": "CustomerChurn[InternetService]", "transformation_label": "String Indexer"}], "feature": "CustomerChurn[InternetService]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "DSL", "max": "Fiber optic", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CustomerChurn[InternetSer..."}, {"transformationsData": [{"feature_label": "CustomerChurn[OnlineSecurity]", "transformation_label": "String Indexer"}], "feature": "CustomerChurn[OnlineSecurity]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "No", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CustomerChurn[OnlineSecur..."}, {"transformationsData": [{"feature_label": "CustomerChurn[OnlineBackup]", "transformation_label": "String Indexer"}], "feature": "CustomerChurn[OnlineBackup]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CustomerChurn[OnlineBacku..."}, {"transformationsData": [{"feature_label": "CustomerChurn[DeviceProtection]", "transformation_label": "String Indexer"}], "feature": "CustomerChurn[DeviceProtection]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "No", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CustomerChurn[DeviceProte..."}, {"transformationsData": [{"feature_label": "CustomerChurn[TechSupport]", "transformation_label": "String Indexer"}], "feature": "CustomerChurn[TechSupport]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "No", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CustomerChurn[TechSupport..."}, {"transformationsData": [{"feature_label": "CustomerChurn[StreamingTV]", "transformation_label": "String Indexer"}], "feature": "CustomerChurn[StreamingTV]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CustomerChurn[StreamingTV..."}, {"transformationsData": [{"feature_label": "CustomerChurn[StreamingMovies]", "transformation_label": "String Indexer"}], "feature": "CustomerChurn[StreamingMovies]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CustomerChurn[StreamingMo..."}, {"transformationsData": [{"feature_label": "CustomerChurn[Contract]", "transformation_label": "String Indexer"}], "feature": "CustomerChurn[Contract]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Month-to-month", "max": "Month-to-month", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CustomerChurn[Contract]"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CustomerChurn[PaperlessBilling]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.7", "stddev": "0.46", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "CustomerChurn[PaperlessBi..."}, {"transformationsData": [{"feature_label": "CustomerChurn[PaymentMethod]", "transformation_label": "String Indexer"}], "feature": "CustomerChurn[PaymentMethod]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Bank transfer (automatic)", "max": "Mailed check", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CustomerChurn[PaymentMeth..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CustomerChurn[MonthlyCharges]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "60.22", "stddev": "19.34", "min": "23.45", "max": "100.25", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CustomerChurn[MonthlyChar..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CustomerChurn[TotalCharges]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "370.74", "stddev": "721.08", "min": "23.45", "max": "4338.6", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CustomerChurn[TotalCharge..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CustomerChurn[Churn]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.62", "stddev": "0.48", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "CustomerChurn[Churn]"}]}))

	#transformationPostExecutionHook(telcochurnpbiautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run telcochurnpbiHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(telcochurnpbiautofe, ["CustomerChurn[SeniorCitizen]", "CustomerChurn[Partner]", "CustomerChurn[Dependents]", "CustomerChurn[tenure]", "CustomerChurn[PhoneService]", "CustomerChurn[PaperlessBilling]", "CustomerChurn[MonthlyCharges]", "CustomerChurn[TotalCharges]", "CustomerChurn[customerID]_stringindexer", "CustomerChurn[gender]_stringindexer", "CustomerChurn[MultipleLines]_stringindexer", "CustomerChurn[InternetService]_stringindexer", "CustomerChurn[OnlineSecurity]_stringindexer", "CustomerChurn[OnlineBackup]_stringindexer", "CustomerChurn[DeviceProtection]_stringindexer", "CustomerChurn[TechSupport]_stringindexer", "CustomerChurn[StreamingTV]_stringindexer", "CustomerChurn[StreamingMovies]_stringindexer", "CustomerChurn[Contract]_stringindexer", "CustomerChurn[PaymentMethod]_stringindexer"], "CustomerChurn[Churn]")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

